# 🦌 ELK Simulation Plumbing 🐺🤓

`IPyElk` can serve as a rich, interactive visualization for complex systems that change
over time.

> This is a _🤓-behind-the-curtain_ notebook: see the
> [demo notebook](08_Simulation_App.ipynb) for what you'd want to show to an audience,
> while still having interactive control of key simulation parameters.

In [ ]:
import json
import math
import re
from datetime import datetime
from datetime import datetime as dt
from datetime import timedelta
from pathlib import Path

import bqplot as B
import bqplot.pyplot as plt
import ipywidgets as W
import networkx
import pandas
import traitlets as T
from bqplot.traits import convert_to_date
from IPython.display import HTML, display
from numpy.random import normal, uniform

import ipyelk
import ipyelk.tools
from ipyelk.contrib.molds.connectors import StraightArrow
from ipyelk.elements import Label, Node, Port, layout_options

## Model Populations

The models will be "dumb" sliders.

In [ ]:
populations = earth, grass, deer, wolves, corpses, poop = [
    W.IntSlider(value=v, description=d, min_value=0, max_value=1000)
    for d, v in {"🌎": 10, "🌱": 60, "🦌": 30, "🐺": 2, "💀": 5, "💩": 5}.items()
]

## Model Time

In [ ]:
speed = W.IntSlider(value=2000, description="🐢🐰", min=1, max=5000)
play = W.Play(max=1000, show_repeat=False)
live_plot = W.Checkbox(True, description="📉", layout=dict(max_width="12em"))
live_elk = W.Checkbox(True, description="🌐", layout=dict(max_width="12em"))
T.link((speed, "value"), (play, "interval"))
date = W.DatePicker(value=datetime.now(), description="📆")
months = [
    (datetime(month=m, year=date.value.year, day=1).strftime("%b"), m)
    for m in range(1, 13)
]
history = None

In [ ]:
def update_history():
    global history

    update = pandas.DataFrame(
        [
            dict(Date=date.value, Name=p.description, Population=p.value)
            for p in populations
        ]
    )
    if history is None:
        history = update
    else:
        history = pandas.concat([history, update])


update_history()

# Model Behaviors

The running of a simulation is entertaining to watch, but even more fun if it can be
configured.

In [ ]:
behaviors = set()
knobs = set()

### 🌱 Behaviors

In [ ]:
sprout_season = W.SelectionRangeSlider(description="🌱📆", value=(3, 10), options=months)
sprout_rate = W.IntSlider(10, description="🌱📶")
knobs |= {sprout_season, sprout_rate}
behaviors |= {(earth, "grows", grass, "up")}

In [ ]:
def grass_tick():
    start, end = sprout_season.value
    if start <= date.value.month <= end:
        grass.value += sprout_rate.value
        earth.value -= 1

### 🦌 Behaviors

In [ ]:
deer_appetite = W.FloatSlider(0.15, description="🦌🍽️")
fawn_season = W.SelectionRangeSlider(description="🦌📆", value=(4, 7), options=months)
fawn_rate = W.FloatSlider(0.25, description="🦌🍼")
knobs |= {deer_appetite, fawn_season, fawn_rate}
behaviors |= {
    (grass, "eaten by", deer, "down"),
    (deer, "make", poop, "up"),
    (deer, "becomes", corpses, "up"),
    (deer, "reproduce", deer, "up"),
}

In [ ]:
def deer_tick():
    if not deer.value:
        return
    start, end = fawn_season.value
    if start <= date.value.month <= end:
        new_deer = (deer.value * fawn_rate.value) or 1
        deer.value += new_deer
    eaten = deer.value * deer_appetite.value
    grass.value -= eaten
    poop.value += eaten
    if not grass.value:
        died = deer.value / 4
        deer.value -= died
        corpses.value += died
    if uniform() > 0.8:
        deer.value -= 1
        corpses.value += 1

### 🐺 Behaviors

In [ ]:
wolf_appetite = W.FloatSlider(0.1, description="🐺🍽️")
pup_season = W.SelectionRangeSlider(description="🐺📆", value=(4, 4), options=months)
pup_rate = W.FloatSlider(0.25, description="🐺🍼")
pack_size = W.IntSlider(6, description="🐺🐺", min_value=1)
knobs |= {wolf_appetite, pup_season, pup_rate, pack_size}
behaviors |= {
    (deer, "eaten by", wolves, "down"),
    (wolves, "make", poop, "up"),
    (corpses, "eaten by", wolves, "down"),
    (wolves, "becomes", corpses, "up"),
    (wolves, "reproduce", wolves, "up"),
}

In [ ]:
def wolf_tick():
    if not wolves.value:
        return
    start, end = pup_season.value
    if start <= date.value.month <= end:
        wolves.value += pup_rate.value
    hungry = wolves.value

    if uniform() > 0.1:
        kills = min(deer.value, wolves.value / pack_size.value)
        deer.value -= kills
        corpses.value += kills
        hungry -= kills * pack_size.value
        poop.value += kills

    if hungry > 0 and corpses.value:
        corpses.value -= 1
        hungry = hungry - pack_size.value
        poop.value += 1

    if hungry > 0:
        corpses.value += 1
        wolves.value -= 1

### 💀 Behaviors

In [ ]:
behaviors |= {
    (corpses, "decomposes into", earth, "up"),
}

In [ ]:
def corpses_tick():
    if not corpses.value:
        return
    decayed = corpses.value / 4
    earth.value += decayed
    corpses.value -= decayed

### 💩 Behaviors

In [ ]:
behaviors |= {
    (poop, "decomposes into", earth, "up"),
}

In [ ]:
def poop_tick():
    if not poop.value:
        return
    decayed = poop.value / 2
    earth.value += decayed
    poop.value -= decayed

## Build the Graph

Get ready to build up the new `networkx` graph from the populations/history.

We'll do some heavy customization, going a little deeper than the
[layout transformer options example](./103_transformer_layout_options.ipynb).

In [ ]:
node_label_options = dict(
    layoutOptions={
        layout_options.NodeLabelPlacement.identifier: "H_CENTER V_CENTER INSIDE"
    }
)

node_options = dict(
    layoutOptions={
        layout_options.NodeSizeConstraints.identifier: "NODE_LABELS PORTS PORT_LABELS",
        layout_options.NodeLabelPlacement.identifier: "H_CENTER V_CENTER",
    }
)

edge_label_options = dict(
    layoutOptions={layout_options.EdgeLabelSideSelection.identifier: "SMART_UP"}
)

We'll draw some "cute" labels, consisting of the emoji repeated a number of times, put
into a box.

In [ ]:
def pretty_label(population):
    txt = population.description * population.value
    d = math.ceil(math.sqrt(len(txt)))
    return re.sub(f"(.{{,{d}}})", r"\1\n", txt).strip().splitlines()

In [ ]:
arrow = StraightArrow(identifier="arrow")


def make_edge_properties(direction):
    return dict(
        properties=dict(
            cssClasses=direction,
            shape=dict(
                end=arrow.identifier,
            ),
        )
    )

In [ ]:
def make_graph():

    graph = networkx.MultiDiGraph()
    [
        graph.add_node(
            p.description,
            id=f"{p.description}",
            labels=[
                Label(id=f"l_{p.description}_{i}", text=line, **node_label_options)
                for i, line in enumerate(pretty_label(p))
            ],
            **node_options,
        )
        # hide a behavior if empty
        for p in populations
        if p.value
    ]

    [
        graph.add_edge(
            eaten.description,
            eater.description,
            id=f"e_{eater.description}_eats_{eaten.description}",
            labels=[
                Label(
                    text=text,
                    id=f"l_{eater.description}_eats_{eaten.description}",
                    **edge_label_options,
                )
            ],
            **make_edge_properties(direction),
        )
        for eaten, text, eater, direction in behaviors
        # hide a behavior if either party is empty
        if eater.value and eaten.value
    ]
    return graph

## Make the Elk

In [ ]:
loader = ipyelk.NXLoader(
    default_root_opts={
        layout_options.EdgeRouting.identifier: "SPLINES",
        layout_options.NodeSizeConstraints.identifier: "NODE_LABELS",
        layout_options.HierarchyHandling.identifier: "INCLUDE_CHILDREN",
    },
)

elk = ipyelk.Diagram(
    source=loader.load(graph=make_graph(), root_id="🏡"),
    layout=dict(display="flex", flex="1"),
    style={
        " rect.elknode": {
            "stroke": "transparent !important",
            "fill": "transparent !important",
        },
        " .elkedge": {"font-weight": "bold"},
        " .up path": {"stroke": "blue !important", "opacity": "0.7"},
        " .up .arrow": {"fill": "blue !important", "opacity": "0.7"},
        " .down path": {"stroke": "red !important", "opacity": "0.7"},
        " .down.elkedge": {"stroke-dasharray": "4", "opacity": "0.7"},
        " .down .arrow": {"fill": "red !important", "opacity": "0.7"},
    },
)

## Handle time changing

In [ ]:
def update_elk(change=None):
    if not live_elk.value:
        return
    elk.source = loader.load(graph=make_graph(), root_id="🏡")

In [ ]:
def tick(change=None):
    date.value = date.value + timedelta(days=7)
    poop_tick()
    corpses_tick()
    grass_tick()
    deer_tick()
    wolf_tick()
    update_history()
    update_elk()
    update_plot()

Wire up observers. Using `lambda` allows for live modification of the behaviors, even
while the simulation is running.

In [ ]:
[p.observe(lambda c: update_elk(), "value") for p in populations]
live_plot.observe(lambda c: update_plot(), "value")
live_elk.observe(lambda c: update_elk(), "value")
play.observe(lambda c: tick(), "value")

## Visualize Time

Graph diagrams aren't great for showing changes over time. To see these, we'll use
[bqplot](https://github.com/bqplot/bqplot).

In [ ]:
COLORS = [
    "rgb(228,26,28)",
    "rgb(55,126,184)",
    "rgb(77,175,74)",
    "rgb(152,78,163)",
    "rgb(255,127,0)",
    "rgb(255,255,51)",
    "rgb(166,86,40)",
    "rgb(247,129,191)",
    "rgb(153,153,153)",
]


def make_bqplot():
    x = B.DateScale()
    y = B.LinearScale()
    scales = dict(x=x, y=y)
    line_opts = dict(scales=scales, stroke_width=1, display_legend=True)

    lines = {
        name: B.Lines(
            x=history[history["Name"] == name]["Date"],
            y=history[history["Name"] == name]["Population"],
            labels=[name],
            colors=[COLORS[i]],
            **line_opts
        )
        for i, name in enumerate(history["Name"])
    }

    ax_x = B.Axis(scale=x, grid_lines="solid", label="Date")
    ax_y = B.Axis(
        scale=y, orientation="vertical", grid_lines="solid", label="Population"
    )

    fig = B.Figure(
        marks=[*lines.values()],
        axes=[ax_x, ax_y],
        title="Populations over time",
        legend_location="top-left",
        layout=dict(max_height="250px"),
    )
    return fig, lines


def update_lines(lines):
    for name in history["Name"]:
        with lines[name].hold_trait_notifications():
            lines[name].x = history[history["Name"] == name]["Date"]
            lines[name].y = history[history["Name"] == name]["Population"]

    return lines


plot, plot_lines = make_bqplot()
update_lines(plot_lines)
plot

DataFrames aren't evented.

In [ ]:
def update_plot():
    update_lines(plot_lines)

## Showtime!

Actually draw the app.

In [ ]:
app = W.HBox(
    [
        W.VBox(
            [
                W.HTML("<h3>Simulation</h3>"),
                play,
                speed,
                date,
                W.HTML("<h3>Populations</h3>"),
                *populations,
                W.HTML("<h3>Parameters</h3>"),
                *sorted(knobs, key=lambda w: w.description),
            ]
        ),
        W.VBox(
            [
                W.HBox(
                    [live_elk, W.HTML("""<h3>Simulation State</h3>""")],
                    layout=dict(align_items="center"),
                ),
                elk,
                W.HBox(
                    [live_plot, W.HTML("""<h3>Populations over time</h3>""")],
                    layout=dict(align_items="center"),
                ),
                plot,
            ],
            layout=dict(flex="1"),
        ),
    ],
    layout=dict(flex="1", height="100%", min_height="80vh"),
)

In [ ]:
if __name__ == "__main__":
    display(app)
    update_plot()
    update_elk()

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).